In [1]:
import json
import random
import pandas as pd
import jsonlines
import warnings
import plotly.graph_objects as go

pd.set_option('display.max_rows', 100)
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

In [2]:
file_path = '/home/carolus/Documents/school/green_ia/data/00_data/00_openfoodfacts_05.jsonl'
num_samples = 60

In [3]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [4]:
df = sample_jsonl_file(file_path, num_samples)

In [ ]:
df.tail(60)

In [ ]:
def load_jsonl_to_dataframe(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df

def create_treemap_for_column(df, column_name):
    if column_name not in df.columns:
        print(f"warning, column: '{column_name}'does not exists")
        return
    data = df[column_name].value_counts().reset_index()
    data.columns = [column_name, 'count']
    fig = go.Figure(go.Treemap(
        labels=data[column_name],
        parents=[""] * len(data),
        values=data['count'],
        textinfo='label+value',  
    ))
    fig.update_layout(
        title=f'treemap column: {column_name}',
        paper_bgcolor='black',  
        plot_bgcolor='grey',   
        font_color='white'      
    )
    fig.show()

In [ ]:
colonnes_a_travailler = ['ecoscore_tags', 'ecoscore_score', 'countries', 'labels_note']  
df = load_jsonl_to_dataframe(file_path)
for column in colonnes_a_travailler:
    create_treemap_for_column(df, column)

In [ ]:
def count_unique_values(jsonl_file_path, colonnes_a_travailler):
    data = []
    with open(jsonl_file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    unique_counts = {}
    for column in colonnes_a_travailler:
        if column in df.columns:
            unique_count = df[column].nunique()
            if pd.api.types.is_numeric_dtype(df[column]):
                min_val = df[column].min()
                max_val = df[column].max()
                unique_counts[column] = {
                    'unique_count': unique_count,
                    'min': min_val,
                    'max': max_val
                }
            else:
                unique_counts[column] = {
                    'unique_count': unique_count
                }
        else:
            unique_counts[column] = 'Column not found in data'
    return unique_counts

In [ ]:
unique_values = count_unique_values(file_path, colonnes_a_travailler)
print(unique_values)